In [ ]:
# ! git clone https://github.com/nbtpj/LMRecTest.git
# import os
# os.chdir('LMRecTest/')
# ! ls
# !mkdir ./cache/
# !cp -R /kaggle/input/lmrec-data/* ./cache/
# !rm -r cache/sampled_prompt_movilens_dataset.hf
# !ls cache

In [ ]:
!python make_data.py 4000

In [ ]:
from env_config import TERM_TO_ESTIMATE, RATINGS, movielen_feat_map

test_scenario = {
    'biography only': ['bio prompt'],
    'bio + top 10 nearest ratings': ['bio prompt', 'history prompt'],
    'bio + 10 nearest ratings + history overview': ['bio prompt', 'history prompt', 'history overview prompt'],
    'bio + 10 nearest ratings + history overview + age-group': ['bio prompt', 'history prompt', 'history overview prompt', 'age-group prompt'],
    'bio + 10 nearest ratings + history overview + age-group + next-category': ['bio prompt', 'history prompt', 'history overview prompt', 'age-group prompt', 'cross-cate prompt'],
}

def film_promt(film_meta: dict) -> dict:
    title = None
    try:
        title = film_meta['movie_title'].decode('utf-8')
    except:
        pass
    categories = []
    try:
        categories = [movielen_feat_map['movie_genres'][k] for k in film_meta['movie_genres']]
    except:
        pass

    return {
        'target prompt': f'The movie named {title} is categorized as {", ".join(categories)}.{TERM_TO_ESTIMATE}'
    }

def map2ctx(rating, test_scenario, targets_ids):
    r = {}
    for name, cols_to_gather in test_scenario.items():
        try:
            r[name] = [rating[col] for col in cols_to_gather]
            r[name] = ' '.join(r[name])
        except:
            pass
    r.update({
        'user_rating': rating['user_rating'],
        'expected': targets_ids.index(rating['movie_id'])
    })
    return r

In [ ]:
RATINGS

In [ ]:
from env_config import device
import torch
from torch import nn

# Test Recommender Scenario
___

In [ ]:
from BARTRec import rank_with_bart
from GPTRec import rank_with_gpt
from datasets import load_from_disk
import numpy as np

    
unified_dataset = load_from_disk('cache/datasets.hf')
sampled_prompt_movilens_dataset = load_from_disk('cache/sampled_prompt_movilens_dataset.hf')
# target_details = unified_dataset['movielens-1m-movies'].map(film_promt)
target_details = sampled_prompt_movilens_dataset.map(film_promt)


targets = target_details['target prompt'] # all possible selections
targets_ids = target_details['movie_id']


context_selection = sampled_prompt_movilens_dataset.map(map2ctx, 
                                                        fn_kwargs = {'test_scenario':test_scenario, 'targets_ids': targets_ids},
                                                        remove_columns=sampled_prompt_movilens_dataset.column_names)
expected = np.array(context_selection['expected'])

In [ ]:
context_selection

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, GPT2Tokenizer, GPT2LMHeadModel
from metrics import *
from tqdm import tqdm

available_selections = target_details['target prompt']
rank_method_names = ['BART', 'GPT']
rank_methods = [rank_with_bart, rank_with_gpt]
models = [BartForConditionalGeneration.from_pretrained('facebook/bart-base').to(device), GPT2LMHeadModel.from_pretrained('gpt2').to(device)]
if torch.cuda.device_count() > 1:
  for i in range(len(models)):
    models[i] = nn.DataParallel(models[i])
tokenizers = [BartTokenizer.from_pretrained('facebook/bart-base'), GPT2Tokenizer.from_pretrained('gpt2')]
tokenizers[-1].add_special_tokens({'pad_token': '[PAD]'})
size = 20
results = []
for scenario in tqdm(test_scenario):
    if scenario in context_selection.column_names:
        contexts = context_selection[scenario]
        for rank_method, model, tokenizer, name in zip(rank_methods, models, tokenizers, rank_method_names):
            predictions = rank_method(model=model, tokenizer=tokenizer,
                                contexts=contexts, available_selections=available_selections,
                                term_to_estimate = TERM_TO_ESTIMATE)
            eval_rs = MMRandR(predictions, expected, size=20)
            results.append({'model': name, 'scenario': scenario, **eval_rs})
len(results)

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df

# Test Rating-prediction Scenario
___

In [ ]:
from BARTRec import rank_with_bart
from GPTRec import rank_with_gpt
from datasets import load_from_disk
import numpy as np

    
unified_dataset = load_from_disk('cache/datasets.hf')
sampled_prompt_movilens_dataset = load_from_disk('cache/sampled_prompt_movilens_dataset.hf')
target_details = unified_dataset['movielens-1m-movies'].map(film_promt)

targets = target_details['target prompt'] # all possible selections
targets_ids = target_details['movie_id']


context_selection = sampled_prompt_movilens_dataset.map(map2ctx, 
                                                        fn_kwargs = {'test_scenario':test_scenario, 'targets_ids': targets_ids},
                                                        remove_columns=sampled_prompt_movilens_dataset.column_names)
expected = np.array(context_selection['user_rating'])


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, GPT2Tokenizer, GPT2LMHeadModel
from metrics import *
from tqdm import tqdm

available_selections = RATINGS
rank_method_names = ['BART', 'GPT']
rank_methods = [rank_with_bart, rank_with_gpt]
models = [BartForConditionalGeneration.from_pretrained('facebook/bart-base').to(device), GPT2LMHeadModel.from_pretrained('gpt2').to(device)]
if torch.cuda.device_count() > 1:
  for i in range(len(models)):
    models[i] = nn.DataParallel(models[i])
tokenizers = [BartTokenizer.from_pretrained('facebook/bart-base'), GPT2Tokenizer.from_pretrained('gpt2')]
tokenizers[-1].add_special_tokens({'pad_token': '[PAD]'})

size = 20
results = []
for scenario in tqdm(test_scenario):
    if scenario in context_selection.column_names:
        contexts = context_selection[scenario]
        for rank_method, model, tokenizer, name in zip(rank_methods, models, tokenizers, rank_method_names):
            predictions = rank_method(model=model, tokenizer=tokenizer,
                                contexts=contexts, available_selections=available_selections,
                                term_to_estimate = TERM_TO_ESTIMATE)
            predictions = predictions[..., 0] + 1 # convert from index to rating
            eval_rs = rmse(predictions, expected)
            results.append({'model': name, 'scenario': scenario, **eval_rs})
len(results)

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df